# Movielens 영화 추천 실습

In [30]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [31]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [32]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [33]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [36]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(100)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
95,97,"Hate (Haine, La) (1995)",Drama
96,98,Shopping (1994),Action|Thriller
97,99,Heidi Fleiss: Hollywood Madam (1995),Documentary
98,100,City Hall (1996),Drama|Thriller


In [37]:
# 영화를 title로 입력할 것이기때문에 title이란 컬럼을 불러와야합니다.
ratings = pd.merge(ratings, movies[['title', 'movie_id']], on='movie_id', how='left')
ratings

,user_id,movie_id,count,timestamp,title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000)
4,1,2355,5,978824291,"Bug's Life, A (1998)"
...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986)
836474,6040,1094,5,956704887,"Crying Game, The (1992)"
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995)
836476,6040,1096,4,956715648,Sophie's Choice (1982)


In [38]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [39]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [40]:
#가장 인기 있는 영화 30개(인기순) 영화 id별 별점(시청횟수)많은 순
movie_count = ratings.groupby('movie_id')['count'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: count, dtype: int64

In [41]:
# 내가 선호하는 영화를 5가지 골라서 rating에 추가
my_favorite = ['minority report', 'mission impossible', 'get out' , 'inception', 'gravity']

my_list = pd.DataFrame({'user_id': ['meissamatar']*5, 'title': my_favorite, 'count':[5]*5})

if not ratings.isin({'user_id':['meissamatar']})['user_id'].any(): 
    ratings = ratings.append(my_list)                         

ratings.tail(10)  

,user_id,movie_id,count,timestamp,title
836473,6040,1090.0,3,956715518.0,Platoon (1986)
836474,6040,1094.0,5,956704887.0,"Crying Game, The (1992)"
836475,6040,562.0,5,956704746.0,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,956715648.0,Sophie's Choice (1982)
836477,6040,1097.0,4,956715569.0,E.T. the Extra-Terrestrial (1982)
0,meissamatar,NaN,5,NaN,minority report
1,meissamatar,NaN,5,NaN,mission impossible
2,meissamatar,NaN,5,NaN,get out
3,meissamatar,NaN,5,NaN,inception
4,meissamatar,NaN,5,NaN,gravity


In [42]:
# 인덱싱
userid_unique = ratings['user_id'].unique()
#movie_unique = ratings['movie_id'].unique()

movie_unique = ratings['title'].unique()

# user, movie indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(userid_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [43]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['meissamatar'])    
print(movie_to_idx['inception'])

6039
3631


In [44]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,count,timestamp,title
0,0,0,5,978300760.0,One Flew Over the Cuckoo's Nest (1975)
1,0,1,3,978302109.0,James and the Giant Peach (1996)
2,0,2,3,978301968.0,My Fair Lady (1964)
3,0,3,4,978300275.0,Erin Brockovich (2000)
4,0,4,5,978824291.0,"Bug's Life, A (1998)"
...,...,...,...,...,...
0,6039,3628,5,NaN,minority report
1,6039,3629,5,NaN,mission impossible
2,6039,3630,5,NaN,get out
3,6039,3631,5,NaN,inception


In [45]:
# csr_matrix
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [46]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
# 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
# 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
# 3. use_gpu : GPU를 사용할 것인지 
# 4. iterations : epochs와 같은 의미입니다.
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=10, dtype=np.float32)

In [47]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [48]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/10 [00:00<?, ?it/s]

In [49]:

meissamatar, inception = user_to_idx['meissamatar'], movie_to_idx['inception']
meissamatar_vector, inception_vector = als_model.user_factors[meissamatar], als_model.item_factors[inception]

print('슝=3')

슝=3


In [50]:
# 내적하는 코드
np.dot(meissamatar_vector, inception_vector)

0.0007646143

In [51]:
getout = movie_to_idx['get out']
getout_vector = als_model.item_factors[getout]
np.dot(meissamatar_vector, getout_vector)

0.0013054933

In [52]:
# (id, 유사도 )
favorite_movie = 'inception'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3631, 0.99999994),
 (3630, 0.9965013),
 (3628, 0.9887395),
 (3632, 0.98788553),
 (3629, 0.9801441),
 (3576, 0.98009604),
 (3574, 0.97990054),
 (3575, 0.97880733),
 (3580, 0.97834903),
 (3573, 0.9773324),
 (3572, 0.97647434),
 (3577, 0.9762067),
 (3578, 0.97604316),
 (3584, 0.97542536),
 (3615, 0.9749763)]

In [53]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['inception',
 'get out',
 'minority report',
 'gravity',
 'mission impossible',
 'War at Home, The (1996)',
 'Century (1993)',
 "Brother's Kiss, A (1997)",
 "Another Man's Poison (1952)",
 'Male and Female (1919)',
 "I Don't Want to Talk About It (De eso no se habla) (1993)",
 'Daens (1992)',
 'Number Seventeen (1932)',
 'Second Best (1994)',
 'Chain of Fools (2000)']

In [54]:
# 비슷한 영화 이름 검색 함수
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [55]:
get_similar_movie('get out')

['get out',
 'inception',
 'gravity',
 'minority report',
 'mission impossible',
 'Century (1993)',
 'War at Home, The (1996)',
 "Brother's Kiss, A (1997)",
 "Another Man's Poison (1952)",
 'Number Seventeen (1932)']

In [56]:
user = user_to_idx['meissamatar']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(126, 0.008087572),
 (22, 0.008046126),
 (1003, 0.007877229),
 (1854, 0.007776116),
 (2609, 0.0076699248),
 (2593, 0.007646013),
 (3131, 0.00755445),
 (2229, 0.007538394),
 (3050, 0.0074307644),
 (243, 0.007404375),
 (3042, 0.0072177406),
 (2044, 0.0071922466),
 (953, 0.007125781),
 (143, 0.0071225553),
 (2894, 0.0071164556),
 (2129, 0.007098563),
 (1570, 0.0070182206),
 (1589, 0.006988917),
 (2273, 0.0069677597),
 (1160, 0.0069624763)]

In [57]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Shakespeare in Love (1998)',
 'Back to the Future (1985)',
 'Blues Brothers 2000 (1998)',
 'Before and After (1996)',
 'Great White Hype, The (1996)',
 'My Favorite Season (1993)',
 'Metroland (1997)',
 'Thieves (Voleurs, Les) (1996)',
 'Jude (1996)',
 'Ghostbusters (1984)',
 'Autumn Sonata (Höstsonaten ) (1978)',
 'Drunken Master (Zui quan) (1979)',
 'Muppet Movie, The (1979)',
 'Gone with the Wind (1939)',
 'When Night Is Falling (1995)',
 'Character (Karakter) (1997)',
 'Dance with Me (1998)',
 'Made in America (1993)',
 'Children of Heaven, The (Bacheha-Ye Aseman) (1997)',
 'Muppets From Space (1999)']